# HTML tags position generation

Con la posición base del primer rectángulo, y aquellas posiciones de los saltos necesarios, calcularemos todos los restantes. Las posiciones vienen dadas por: horizontalizq, verticalarriba, horizontalder, verticalabajo

Input parameters: 
* gr_col: numero de 'columnas de grupos'
* gr_row: numero de 'filas de grupos'
* col: numero de columnas/grupo
* row: numero de filas/grupo 

In [8]:
import numpy as np

In [9]:
def mostrar(path):
    with open(path, 'r') as file:
        imgmap = file.read()
    print(imgmap)

## Creación de la clase

In [10]:
class mapeador():
    
    def __init__(self, path_input, path_output, gr_col, gr_row, col, row):
        self.path_input = path_input
        self.path_output = path_output
        self.gr_col = gr_col
        self.gr_row = gr_row
        self.col = col
        self.row = row
        
        self.importar_map()
        
    def importar_map(self):
        with open(self.path_input, 'r') as file:
            self.imgmap = file.read()
    
    def coordenadas(self):
        #Conseguimos el diccionario con las cordenadas del rectángulo inicial y los saltos necesarios
        coords_dict = {}
        split = [split.split('\" alt=\"') for split in self.imgmap.split('coords=\"')[1:]]
        
        for coords in split:
            alt = coords[1].split('\"  nohref')[0] #Careful: dos espacios entre \" y nohref
            coords_dict[int(alt)] = coords[0]
        
        return coords_dict
        
    def distancias(self):
        #Calculamos las distancias al rectángulo inicial (base)
        coords_dict = self.coordenadas()
        
        dist_dict = {}
        pos_base = [int(pos) for pos in coords_dict[1].split(",")]
        
        for entry in coords_dict.keys():
            if entry == 1:
                continue
            pos = [int(pos) for pos in coords_dict[entry].split(",")]
            dist_dict[entry] = np.subtract(pos,pos_base)
        
        dist_good = {entry: list(dist_dict[entry][:2]) + list(dist_dict[entry][:2]) for entry in dist_dict.keys()} # Nos quedamos con las dos primeras entradas, y las repetimos.
        
        return dist_good, pos_base
        
    def todas_posiciones(self):
        #obtenemos un diccionario alt <---> posiciones de los rectángulos
        dist_good, pos_base = self.distancias()
    
        alt = 1
        all_pos = {}
        all_pos[1] = np.array(pos_base)

        def calcular_posicion(alt,i,j,k,l):
            change = np.multiply(i,dist_good[2]) + np.multiply(j,dist_good[3]) + np.multiply(k,dist_good[7]) 
            pos = np.add(pos_base, change)
            alt += 1
            return alt, pos 

        for l in range(0,gr_row):
            for k in range(0,gr_col):
                for j in range(0,col):
                    for i in range(0,row):
                        if i == 0 and j == 0 and k == 0 and l == 0:
                            continue
                        alt, pos = calcular_posicion(alt,i,j,k,l)
                        all_pos[alt] = pos
                        
        return all_pos
    
    def preparar_output(self):
        #Eliminar los tags que ahora sustituiremos
        splitmap = self.imgmap.split('\n')
        finalmap = [] 
        for split in splitmap:
            if split.find('<area'):
                finalmap.append(split)

        return finalmap
    
    def generar_y_guardar(self):
        # Insertar los tags en la lista que irá al fichero
        all_pos = self.todas_posiciones()
        finalmap = self.preparar_output()
        
        for entry in all_pos.keys():
            intlist = all_pos[entry].tolist()
            strlist = [str(num) for num in intlist]
            val = ",".join(strlist)
            tag = f'<area shape="rect" coords="{val}" alt="{entry}"  nohref="nohref" />'
            finalmap.insert(-2,tag)
        
        with open(self.path_output, 'w') as file:
            file.write('\n'.join(finalmap))
        
        print("alright!")

### Caso 1: switch SX6536

In [11]:
#Detalles switch:
gr_col = 3
gr_row = 1
col = 3
row = 2

In [12]:
#Input/output:
path_input = 'imagenes_y_maps/[SX6536_detalle] (imported).map'
path_output = 'imagenes_y_maps/SX6536_detalle.map'

In [13]:
switchSX6536 = mapeador(path_input, path_output, gr_col, gr_row, col, row)

In [7]:
switchSX6536.generar_y_guardar()

alright!
